In [ ]:
#pip install ruamel-yaml
!pip install nltk textstat
!pip install PyPDF2
!pip install pycryptodome
!pip install spacy
!python -m spacy download en_core_web_sm
!pip install ipywidgets
!pip install matplotlib
!pip install textblob

In [ ]:
#calling the necessary library
import os
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from PyPDF2 import PdfReader
import textstat
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from IPython.display import clear_output, display
import ipywidgets as widgets
import string
from tkinter import Tk, filedialog
from nltk.stem import WordNetLemmatizer
import re

# Import additional libraries for clustering
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import seaborn as sns
from textblob import TextBlob


In [ ]:
# Download necessary NLTK resources
nltk.download('stopwords')
nltk.download('punkt')
#nltk.download('wordnet')

# Code to select file

In [ ]:
file_paths = [] 

def select_files(b):
    global file_paths
    root = Tk()
    root.withdraw()  
    file_paths = filedialog.askopenfilenames()  
    print(file_paths) 

fileselect = widgets.Button(description="Select Files")
fileselect.on_click(select_files)

display(fileselect)

print("Please select the PDF files you wish to process.")

# Coding to check on optimum cluster number

In [ ]:
# Function to extract text from a PDF file given a path
def extract_text_from_pdf(file_path):
    with open(file_path, 'rb') as f:
        pdf = PdfReader(f)
        text = ""
        for page in pdf.pages:
            text += page.extract_text() or ""  # Ensure we concatenate strings
        return text

# Function to calculate averages of the scores
def calculate_averages(df):
    return df.mean()

# Function to find the optimal number of clusters using the Elbow Method and display WCSS table and plot
def find_optimal_clusters(data, max_k=10):
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(data)
    wcss = []
    
    for k in range(1, max_k + 1):
        kmeans = KMeans(n_clusters=k, random_state=42)
        kmeans.fit(scaled_data)
        wcss.append(kmeans.inertia_)
    
    # Display WCSS values in a table
    wcss_df = pd.DataFrame({'Number of Clusters (k)': range(1, max_k + 1), 'WCSS': wcss})
    display(wcss_df)
    
    # Plot the Elbow graph
    plt.figure(figsize=(10, 6))
    plt.plot(range(1, max_k + 1), wcss, marker='o', linestyle='--')
    plt.xlabel('Number of Clusters (k)')
    plt.ylabel('Within-Cluster Sum of Squares (WCSS)')
    plt.title('Elbow Method For Optimal Number of Clusters')
    plt.grid(True)
    plt.show()

# Process the selected files
def process_files(file_paths, n_clusters=3):
    clear_output(wait=True)
    results = []
    for file_path in file_paths:
        # Only process .pdf files
        if file_path.endswith('.pdf'):
            # Read the file content and extract text
            text = extract_text_from_pdf(file_path)

            # Tokenize the text into sentences and words
            sentences = sent_tokenize(text)
            words = word_tokenize(text)

            # Calculate readability scores
            fk_grade_level = textstat.flesch_kincaid_grade(text)
            gunning_fog = textstat.gunning_fog(text)
            flesch_reading_ease = textstat.flesch_reading_ease(text)
            smog_index = textstat.smog_index(text)
            
            # Append results for the current file
            results.append({
                'File': os.path.basename(file_path),
                'Flesch-Kincaid Grade Level': fk_grade_level,
                'Gunning Fog Index': gunning_fog,
                'Flesch Reading Ease': flesch_reading_ease,
                'SMOG Index': smog_index,
            })
    
    # Convert the results to a DataFrame
    df = pd.DataFrame(results)
    
    # Calculate averages
    averages = calculate_averages(df.iloc[:, 1:])
    
    # Perform clustering
    features = df.iloc[:, 1:]  # Use all columns except the 'File' column for clustering
    find_optimal_clusters(features, max_k=10)
    
    # Save the DataFrame with cluster labels
    global clustered_data
    clustered_data = df.copy()

# Add a button to trigger file processing after selection
process_button = widgets.Button(description="Process Files")
process_button.on_click(lambda b: process_files(file_paths))
display(process_button)

 # Coding to apply Readability Formula and Clustering

In [ ]:
# Function to extract text from a PDF file given a path
def extract_text_from_pdf(file_path):
    with open(file_path, 'rb') as f:
        pdf = PdfReader(f)
        text = ""
        for page in pdf.pages:
            text += page.extract_text() or ""  # Ensure we concatenate strings
        return text

# Function to calculate averages of the scores
def calculate_averages(df):
    return df.mean()


# Process the selected files
def process_files(file_paths, n_clusters=3):
    clear_output(wait=True)
    results = []
    for file_path in file_paths:
        # Only process .pdf files
        if file_path.endswith('.pdf'):
            # Read the file content and extract text
            text = extract_text_from_pdf(file_path)

            # Tokenize the text into sentences and words
            sentences = sent_tokenize(text)
            words = word_tokenize(text)

            # Calculate readability scores
            fk_grade_level = textstat.flesch_kincaid_grade(text)
            gunning_fog = textstat.gunning_fog(text)
            flesch_reading_ease = textstat.flesch_reading_ease(text)
            smog_index = textstat.smog_index(text)


            # Perform sentiment analysis
            blob = TextBlob(text)
            polarity, subjectivity = blob.sentiment.polarity, blob.sentiment.subjectivity
            
            # Append results for the current file
            results.append({
                'File': os.path.basename(file_path),
                'Flesch-Kincaid Grade Level': fk_grade_level,
                'Gunning Fog Index': gunning_fog,
                'Flesch Reading Ease': flesch_reading_ease,
                'SMOG Index': smog_index,
            })
    
    # Convert the results to a DataFrame and display it
    df = pd.DataFrame(results)
    
    # Calculate averages
    averages = calculate_averages(df.iloc[:, 1:])
    
    # Perform clustering
    features = df.iloc[:, 1:-2]  # Exclude ASL and ASW from clustering features
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(features)
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    kmeans.fit(scaled_features)
    df['Cluster'] = kmeans.labels_
    
    # Set pandas options to display all rows
    pd.set_option('display.max_rows', None)
    # Display the DataFrame
    display(df)
    
    # Save the DataFrame with cluster labels
    global clustered_data
    clustered_data = df.copy()

    # Display averages
    print("Average Scores:")
    display(averages)
    
    
# Add a button to trigger file processing after selection
process_button = widgets.Button(description="Process Files")
process_button.on_click(lambda b: process_files(file_paths))
display(process_button)

 # Coding for statistical summary

In [ ]:
# Function to display statistical summary
def display_statistical_summary(df):
    summary = df.describe().transpose()
    display(summary)

# Function to plot histograms for readability scores with mean labels
def plot_histograms(df):
    fig, axes = plt.subplots(2, 2, figsize=(15, 15))
    axes = axes.flatten()

    # Exclude 'File', 'Cluster', and 'Dale-Chall Score' columns
    metrics_to_plot = [
        'Flesch-Kincaid Grade Level',
        'Gunning Fog Index',
        'Flesch Reading Ease',
        'SMOG Index'
    ]
    
    for idx, col in enumerate(metrics_to_plot):
        sns.histplot(df[col], ax=axes[idx], kde=True)
        mean_val = df[col].mean()
        axes[idx].axvline(mean_val, color='r', linestyle='--')
        axes[idx].text(mean_val, axes[idx].get_ylim()[1] * 0.9, f'Mean: {mean_val:.2f}', color='r')
        axes[idx].set_title(f'Histogram of {col}')
    
    plt.tight_layout()
    plt.show()

# Example usage (make sure df is loaded)
# Assume df is the DataFrame created from the process_files function

# Load your DataFrame (clustered_data) and use it for analysis
def load_and_analyze():
    global clustered_data  # Assuming clustered_data is the DataFrame from the previous part
    df = clustered_data.copy()  # Make sure this DataFrame is already created

    # Display statistical summary
    display_statistical_summary(df.iloc[:, 1:])
    
    # Plot histograms
    plot_histograms(df)

# Button to trigger analysis
analyze_button = widgets.Button(description="Analyze Data")
analyze_button.on_click(lambda b: load_and_analyze())
display(analyze_button)


 # Coding to check number of documents in each cluster

In [ ]:
# Function to calculate and display the number of documents in each cluster
def calculate_and_display_cluster_counts():
    if 'clustered_data' in globals():
        cluster_counts = clustered_data['Cluster'].value_counts().sort_index()
        print("Number of documents in each cluster:")
        print(cluster_counts)
    else:
        print("Data not found. Please run the data processing cell first.")

# Add a button to trigger cluster count calculation
cluster_count_button = widgets.Button(description="Calculate Cluster Counts")
cluster_count_button.on_click(lambda b: calculate_and_display_cluster_counts())
display(cluster_count_button)

 # Coding to visualise Box-Plot by each Cluster

In [ ]:
# Make sure this cell is run after the data processing cell
def generate_boxplots(output_dir='output_plots'):
    if 'clustered_data' in globals():
        scores = ['Flesch-Kincaid Grade Level', 'Gunning Fog Index', 'Flesch Reading Ease', 'SMOG Index']
        for score in scores:
            plt.figure(figsize=(10, 6))
            ax = sns.boxplot(x='Cluster', y=score, data=clustered_data)
            plt.title(f'Boxplot of {score} by Cluster')

            # Calculate means and add them as text annotations
            means = clustered_data.groupby('Cluster')[score].mean()
            for index, mean in enumerate(means):
                ax.text(index, mean, f'{mean:.2f}', color='black', ha="center")

            # Check if output directory exists, if not, create it
            if not os.path.exists(output_dir):
                os.makedirs(output_dir)

            # Save the figure before showing it
            plt.savefig(f"{output_dir}/{score.replace(' ', '_')}_Boxplot.png")

            # Show the plot
            plt.show()
    else:
        print("Data not found. Please run the data processing cell first.")

# Call the function to generate and save boxplots
generate_boxplots()

 # Coding for summary statistics by cluster

In [ ]:
import pandas as pd
import numpy as np

# Function to detect outliers using IQR method
def detect_outliers(data):
    q1 = np.percentile(data, 25)
    q3 = np.percentile(data, 75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    return ((data < lower_bound) | (data > upper_bound)).sum()

# Function to generate and display enhanced summary statistics for each cluster
def generate_and_display_summary_statistics():
    if 'clustered_data' in globals():
        summary_statistics = clustered_data.groupby('Cluster').agg({
            'Flesch-Kincaid Grade Level': ['mean', 'median', 'std', 'min', 'max', detect_outliers],
            'Gunning Fog Index': ['mean', 'median', 'std', 'min', 'max', detect_outliers],
            'Flesch Reading Ease': ['mean', 'median', 'std', 'min', 'max', detect_outliers],
            'SMOG Index': ['mean', 'median', 'std', 'min', 'max', detect_outliers]
        }).reset_index()

        # Flatten the MultiIndex columns
        summary_statistics.columns = [' '.join(col).strip() for col in summary_statistics.columns.values]
        
        # Rename the outliers column for clarity
        summary_statistics = summary_statistics.rename(columns={
            'Flesch-Kincaid Grade Level detect_outliers': 'Flesch-Kincaid Grade Level Outliers',
            'Gunning Fog Index detect_outliers': 'Gunning Fog Index Outliers',
            'Flesch Reading Ease detect_outliers': 'Flesch Reading Ease Outliers',
            'SMOG Index detect_outliers': 'SMOG Index Outliers'
        })

        # Display the summary statistics
        display(summary_statistics)
    else:
        print("Data not found. Please run the data processing cell first.")

# Call the function to generate and display summary statistics
generate_and_display_summary_statistics()


In [ ]:
!pip install xgboost
!pip install mlxtend

In [ ]:
import xgboost as xgb
from mlxtend.classifier import StackingClassifier
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

 # Coding to check optimum number for KNN

In [ ]:
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier

# Range of n values to test
n_neighbors = range(1, 10)

# List to store cross-validation scores
cv_scores = []

for k in n_neighbors:
    knn = KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(knn, X_train, y_train, cv=5, scoring='accuracy')
    cv_scores.append(scores.mean())

# Find the best k
best_k = n_neighbors[cv_scores.index(max(cv_scores))]
print(f"Best number of neighbors: {best_k}")

# Plot the results
plt.figure(figsize=(8, 6))  # Adjust figure size for better visibility
plt.plot(n_neighbors, cv_scores, marker='o', linestyle='-', color='b')
plt.xlabel('Number of Neighbors K')
plt.ylabel('Cross-Validated Accuracy')
plt.title('KNN Varying Number of Neighbors')
plt.annotate(f'Best k = {best_k}', 
             xy=(best_k, max(cv_scores)), 
             xytext=(best_k + 0.3, max(cv_scores) - 0.03),
             arrowprops=dict(facecolor='black', arrowstyle='->', lw=1.5),
             fontsize=10, ha='center')
plt.grid(True)

# Save the plot
plot_path = 'C:/Users/User/OneDrive/Documents/Along/Master/STQD6889 Capstone Project/Part 2/knn_varying_neighbors.png'
plt.savefig(plot_path)
plt.show()

# Display the path where the plot is saved
plot_path

 # Coding for classification

In [ ]:

# Define the target and feature columns
target = 'Cluster'
features = clustered_data.drop(columns=[target, 'File'])  # Exclude 'File' and 'Cluster' columns

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, clustered_data[target], test_size=0.2, random_state=42)

# Train a Decision Tree Classifier
decision_tree = DecisionTreeClassifier(random_state=42)
decision_tree.fit(X_train, y_train)

# Train a Random Forest Classifier
random_forest = RandomForestClassifier(random_state=42)
random_forest.fit(X_train, y_train)

# Train a K-Nearest Neighbors Classifier
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

# Train an XGBoost Classifier
xgboost = xgb.XGBClassifier(random_state=42)
xgboost.fit(X_train, y_train)

# Predict and evaluate the Decision Tree Classifier
y_pred_tree = decision_tree.predict(X_test)
tree_accuracy = accuracy_score(y_test, y_pred_tree)
tree_report = classification_report(y_test, y_pred_tree)

# Predict and evaluate the Random Forest Classifier
y_pred_forest = random_forest.predict(X_test)
forest_accuracy = accuracy_score(y_test, y_pred_forest)
forest_report = classification_report(y_test, y_pred_forest)

# Predict and evaluate the K-Nearest Neighbors Classifier
y_pred_knn = knn.predict(X_test)
knn_accuracy = accuracy_score(y_test, y_pred_knn)
knn_report = classification_report(y_test, y_pred_knn)

# Predict and evaluate the XGBoost Classifier
y_pred_xgb = xgboost.predict(X_test)
xgb_accuracy = accuracy_score(y_test, y_pred_xgb)
xgb_report = classification_report(y_test, y_pred_xgb)


# Display the results
print("Decision Tree Classifier Accuracy:", tree_accuracy)
print("Decision Tree Classification Report:")
print(tree_report)

print("\nRandom Forest Classifier Accuracy:", forest_accuracy)
print("Random Forest Classification Report:")
print(forest_report)

print("K-Nearest Neighbors Classifier Accuracy:", knn_accuracy)
print("K-Nearest Neighbors Classification Report:")
print(knn_report)

print("\nXGBoost Classifier Accuracy:", xgb_accuracy)
print("XGBoost Classification Report:")
print(xgb_report)


#paper support kenapa kita pilih random forest (bagging), decision tree, KNN - traditional method
#try XGBoost (boosting), stack generalisation machine learning (combination) - ensemble method
#paper support accuracy, precision, recall, f1-score
#cari features importance among dalam score/characteristic, random forest & decision tree almost the same. tapi KNN lain


 # Coding to check on feature importance

In [ ]:
import pandas as pd
from sklearn.inspection import permutation_importance

# Random Forest Feature Importance
rf_importances = random_forest.feature_importances_
rf_importance_df = pd.DataFrame({'Feature': features.columns, 'Importance': rf_importances})
rf_importance_df = rf_importance_df.sort_values(by='Importance', ascending=False)
print("Random Forest Feature Importances:")
print(rf_importance_df)

# Decision Tree Feature Importance
dt_importances = decision_tree.feature_importances_
dt_importance_df = pd.DataFrame({'Feature': features.columns, 'Importance': dt_importances})
dt_importance_df = dt_importance_df.sort_values(by='Importance', ascending=False)
print("\nDecision Tree Feature Importances:")
print(dt_importance_df)

# K-Nearest Neighbors Feature Importance using Permutation Importance
knn_importance_result = permutation_importance(knn, X_test, y_test, n_repeats=10, random_state=42)
knn_importance_df = pd.DataFrame({'Feature': features.columns, 'Importance': knn_importance_result.importances_mean})
knn_importance_df = knn_importance_df.sort_values(by='Importance', ascending=False)
print("\nK-Nearest Neighbors Feature Importances:")
print(knn_importance_df)

# XGBoost Feature Importance
xgb_importances = xgboost.feature_importances_
xgb_importance_df = pd.DataFrame({'Feature': features.columns, 'Importance': xgb_importances})
xgb_importance_df = xgb_importance_df.sort_values(by='Importance', ascending=False)
print("\nXGBoost Feature Importances:")
print(xgb_importance_df)
